In [1]:
import pandas as pd

In [2]:
df_oh = pd.read_pickle('./data_frames/dense_acs_mm_oh.pkl')

In [3]:
df_oh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1569188 entries, 0 to 1630166
Columns: 898 entries, WKHP to RAC1P_9
dtypes: float64(3), uint8(895)
memory usage: 1.4 GB


In [4]:
df_oh.head()

,WKHP,AGEP,PINCP,OCCP_10,OCCP_20,OCCP_40,OCCP_51,OCCP_52,OCCP_60,OCCP_101,...,MAR_5,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9
0,0.295918,0.025316,0.039472,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0.397959,0.037975,0.030967,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,0.173469,0.025316,0.030467,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0.051020,0.215190,0.053479,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0.091837,0.025316,0.009455,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [5]:
from build_all_models import *

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_oh.drop('PINCP', axis=1), 
                                                    df_oh['PINCP'], test_size = 0.2,
                                                    shuffle=False)

In [7]:
X_dat = df_oh.drop('PINCP', axis=1)
y_dat = df_oh['PINCP']

In [8]:
import numpy as np
A_t = np.load('./data_frames/nparrays/allstates2021_20230724-233202_unshuffled__dropped_.npy')

In [9]:
# np.all(A_t1 == A_t2)

In [10]:
np.all(df_oh['RAC1P_2'] == A_t[:, 3])

True

In [11]:
dirname = 'onehot_encoded'
filename = 'linearmodelRidge'

# Online Linear Model, always active

In [13]:
from OnlineLinModel_alwaysactive_implementable import *
expert = OnlineLinearModel_alwaysactive(l2_pen=1.0)

In [36]:
# bbass = build_baseline_alwayson(dirname, filename, X_train.iloc[:1000], \
#                                   y_train.iloc[:1000], A_t[:1000], expert, l2_pen=0.0)

bbass = build_baseline_alwayson(dirname, filename, X_dat, y_dat, A_t, expert, l2_pen=0.0)

100%|██████████| 1569188/1569188 [34:23<00:00, 760.37it/s]


In [46]:
online_linear_always = joblib.load('./onehot_encoded/models/linear_sepbuild/linearmodelLS20230820-233239_l2pen=0.0.pkl')

In [47]:
for gnum in range(A_t.shape[1]):
    print(f'''group num {gnum}, {online_linear_always.cumloss_groupwise_oridge[gnum][-1]} ''')

group num 0, 20913.693905339678 
group num 1, 14469.89144712689 
group num 2, 24725.04631404486 
group num 3, 2429.3549540855474 
group num 4, 243.37326141763626 
group num 5, 16.449316015757756 
group num 6, 37.849923378080334 
group num 7, 3043.8553935423542 
group num 8, 57.87773421753203 
group num 9, 1542.936266651879 
group num 10, 3286.842189112958 
group num 11, 35383.58535246591 


# Anh new pipeline

In [14]:
from OnlineLinExpert import *
experts = [Online_linearexpert(l2_pen=1.0) for _ in range(A_t.shape[1])]

In [15]:
Anh_new = build_Anh(dirname, filename, X_dat, \
                    y_dat, A_t, \
                    experts
         )

100%|██████████| 1569188/1569188 [1:41:11<00:00, 258.45it/s]


In [2]:
import joblib
Anh = joblib.load('./onehot_encoded/models/Anh_sepbuild/linearmodelRidge20230821-004744.pkl')

In [4]:
for gnum in range(12):
    print(f'''group num {gnum}, {Anh.cuml_loss_curve[gnum][-1]} ''')

group num 0, 30475.73819020837 
group num 1, 22227.669286399614 
group num 2, 37181.656706009446 
group num 3, 3326.114008403669 
group num 4, 325.81384868452835 
group num 5, 19.450365142991945 
group num 6, 52.192409915398414 
group num 7, 4709.026106546749 
group num 8, 73.4781794658253 
group num 9, 2063.3343254242445 
group num 10, 4952.341527015159 
group num 11, 52703.407476607375 


In [17]:
len(Anh.cuml_loss_curve)

12